# Mimic III Dataset

In [ ]:
!pip install -qU pinecone-client langchain openai sqlalchemy tiktoken cohere ipython-sql

In [1]:
%load_ext sql

Directly download files in colab
!wget -r -N -c -np https://physionet.org/files/mimiciii-demo/1.4/

In [12]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, text
import pandas as pd
import os

# Create a SQLAlchemy engine with an in-memory SQLite database
engine = create_engine("sqlite:///mimic3.db",pool_pre_ping=True)
engine.connect()

In [10]:
csv_directory = 'medmind_langserve/mimic-iii-clinical-database-demo-1.4'
# List to store table objects
tables = []

# Iterate over CSV files in the directory
for file_name in os.listdir(csv_directory):
  if file_name.endswith(".csv"):
    # Use the CSV file name (without extension) as the table name
    table_name = os.path.splitext(file_name)[0]
    tables.append(table_name)
    # Read CSV file into a pandas DataFrame
    df = pd.read_csv(os.path.join(csv_directory, file_name), low_memory=False)
    print(df.head())

    # Insert data into the table
    df.to_sql(con=engine,name = table_name,  if_exists='replace', index=False)

   row_id  subject_id  hadm_id            admittime            dischtime  \
0   12258       10006   142345  2164-10-23 21:09:00  2164-11-01 17:15:00   
1   12263       10011   105331  2126-08-14 22:32:00  2126-08-28 18:59:00   
2   12265       10013   165520  2125-10-04 23:36:00  2125-10-07 15:13:00   
3   12269       10017   199207  2149-05-26 17:19:00  2149-06-03 18:42:00   
4   12270       10019   177759  2163-05-14 20:43:00  2163-05-15 12:00:00   

             deathtime admission_type         admission_location  \
0                  NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
1  2126-08-28 18:59:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
2  2125-10-07 15:13:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
3                  NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
4  2163-05-15 12:00:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   

  discharge_location insurance language  religion marital_status  \
0   HOME HEALTH CARE  Medicare      NaN  CATHOLIC      SEPARATED  

OperationalError: (sqlite3.OperationalError) database is locked
[SQL: 
DROP TABLE "ADMISSIONS"]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [15]:
csv_directory = 'medmind_langserve/mimic-iii-clinical-database-demo-1.4'
# List to store table objects
tables = []

# Iterate over CSV files in the directory
for file_name in os.listdir(csv_directory):
  if file_name.endswith(".csv"):
    # Use the CSV file name (without extension) as the table name
    table_name = os.path.splitext(file_name)[0]
    tables.append(table_name)

In [16]:
tables, len(tables)

(['ADMISSIONS',
  'CALLOUT',
  'CAREGIVERS',
  'CHARTEVENTS',
  'CPTEVENTS',
  'DATETIMEEVENTS',
  'DIAGNOSES_ICD',
  'DRGCODES',
  'D_CPT',
  'D_ICD_DIAGNOSES',
  'D_ICD_PROCEDURES',
  'D_ITEMS',
  'D_LABITEMS',
  'ICUSTAYS',
  'INPUTEVENTS_CV',
  'INPUTEVENTS_MV',
  'LABEVENTS',
  'MICROBIOLOGYEVENTS',
  'OUTPUTEVENTS',
  'PATIENTS',
  'PRESCRIPTIONS',
  'PROCEDUREEVENTS_MV',
  'PROCEDURES_ICD',
  'SERVICES',
  'TRANSFERS'],
 25)

In [17]:
# Confirm data insertion
for table_name in tables:
  result_data = pd.read_sql(text(f"SELECT * FROM '{table_name}' LIMIT 5"),engine.connect())
  print(f"\nContents of table '{table_name}':")
  print(result_data)


Contents of table 'ADMISSIONS':
   row_id  subject_id  hadm_id            admittime            dischtime  \
0   12258       10006   142345  2164-10-23 21:09:00  2164-11-01 17:15:00   
1   12263       10011   105331  2126-08-14 22:32:00  2126-08-28 18:59:00   
2   12265       10013   165520  2125-10-04 23:36:00  2125-10-07 15:13:00   
3   12269       10017   199207  2149-05-26 17:19:00  2149-06-03 18:42:00   
4   12270       10019   177759  2163-05-14 20:43:00  2163-05-15 12:00:00   

             deathtime admission_type         admission_location  \
0                 None      EMERGENCY       EMERGENCY ROOM ADMIT   
1  2126-08-28 18:59:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
2  2125-10-07 15:13:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
3                 None      EMERGENCY       EMERGENCY ROOM ADMIT   
4  2163-05-15 12:00:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   

  discharge_location insurance language  religion marital_status  \
0   HOME HEALTH CARE  Medicare   

In [ ]:
engine.

## Initializing agent

In [17]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities import SQLDatabase

In [18]:
db = SQLDatabase(engine)

### Initializing LLM

In [54]:
from getpass import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

In [55]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0
)

In [60]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(memory_key="chat_history",llm=llm)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [68]:
custom_prefix = """
Tables are linked by identifiers which usually have the suffix ‘ID’. For example, SUBJECT_ID refers to a unique patient, HADM_ID refers to a unique admission to the hospital, and ICUSTAY_ID refers to a unique admission to an intensive care unit.

Charted events such as notes, laboratory tests, and fluid balance are stored in a series of ‘events’ tables. For example the OUTPUTEVENTS table contains all measurements related to output for a given patient, while the LABEVENTS table contains laboratory test results for a patient.

Tables prefixed with ‘D_’ are dictionary tables and provide definitions for identifiers. For example, every row of CHARTEVENTS is associated with a single ITEMID which represents the concept measured, but it does not contain the actual name of the measurement. By joining CHARTEVENTS and D_ITEMS on ITEMID, it is possible to identify the concept represented by a given ITEMID.

Broadly speaking, five tables are used to define and track patient stays: ADMISSIONS; PATIENTS; ICUSTAYS; SERVICES; and TRANSFERS. Another five tables are dictionaries for cross-referencing codes against their respective definitions: D_CPT; D_ICD_DIAGNOSES; D_ICD_PROCEDURES; D_ITEMS; and D_LABITEMS. The remaining tables contain data associated with patient care, such as physiological measurements, caregiver observations, and billing information.

In some cases it would be possible to merge tables—for example, the D_ICD_PROCEDURES and CPTEVENTS tables both contain detail relating to procedures and could be combined—but our approach is to keep the tables independent for clarity, since the data sources are significantly different. Rather than combining the tables within MIMIC data model, we suggest researchers develop database views and transforms as appropriate.

Make sure to use tables prefixed with ‘D_’ as much as possible.
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    prefix=custom_prefix,

)

In [41]:
agent.invoke("how many total patients are there")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `{'table_names': 'PATIENTS'}`



CREATE TABLE "PATIENTS" (
	row_id BIGINT, 
	subject_id BIGINT, 
	gender TEXT, 
	dob TEXT, 
	dod TEXT, 
	dod_hosp TEXT, 
	dod_ssn TEXT, 
	expire_flag BIGINT
)

/*
3 rows from PATIENTS table:
row_id	subject_id	gender	dob	dod	dod_hosp	dod_ssn	expire_flag
9467	10006	F	2094-03-05 00:00:00	2165-08-12 00:00:00	2165-08-12 00:00:00	2165-08-12 00:00:00	1
9472	10011	F	2090-06-05 00:00:00	2126-08-28 00:00:00	2126-08-28 00:00:00	None	1
9474	10013	F	2038-09-03 00:00:00	2125-10-07 00:00:00	2125-10-07 00:00:00	2125-1

{'input': 'how many total patients are there',
 'output': 'There are 100 total patients.'}

In [36]:
agent.invoke("Are there any patients who had heart disease diagnosis more than 2 times give their Subject id")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `{'table_names': 'DIAGNOSES_ICD'}`
responded: The DIAGNOSES_ICD table seems to be the most relevant for this query as it contains the ICD codes for diagnoses. Let's check the schema of this table.


CREATE TABLE "DIAGNOSES_ICD" (
	row_id BIGINT, 
	subject_id BIGINT, 
	hadm_id BIGINT, 
	seq_num BIGINT, 
	icd9_code TEXT
)

/*
3 rows from DIAGNOSES_ICD table:
row_id	subject_id	hadm_id	seq_num	icd9_code
112344	10006	142345	1	99591
112345	10006	142345	2	99662
112346	10006	142345	3	5672
*/
Invoking: `sql_db_query_checker` with `{'query': "

{'input': 'Are there any patients who had heart disease diagnosis more than 2 times give their Subject id',
 'output': 'The patients with Subject IDs 10042, 10088, 43827, and 43927 have had a heart disease diagnosis more than 2 times.'}

In [58]:
agent.invoke("How many total admissions are present of 10013")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `{'table_names': 'ADMISSIONS'}`
responded: The ADMISSIONS table seems to be the most relevant for this query. Let's check its schema.


CREATE TABLE "ADMISSIONS" (
	row_id BIGINT, 
	subject_id BIGINT, 
	hadm_id BIGINT, 
	admittime TEXT, 
	dischtime TEXT, 
	deathtime TEXT, 
	admission_type TEXT, 
	admission_location TEXT, 
	discharge_location TEXT, 
	insurance TEXT, 
	language TEXT, 
	religion TEXT, 
	marital_status TEXT, 
	ethnicity TEXT, 
	edregtime TEXT, 
	edouttime TEXT, 
	diagnosis TEXT, 
	hospital_expire_flag BIGINT, 
	has_chart

{'input': 'How many total admissions are present of 10013',
 'output': 'The patient with the ID 10013 has been admitted to the hospital once.'}

In [65]:
agent.invoke("Can you identify all subject id with sepsis")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `{'table_names': 'DIAGNOSES_ICD'}`
responded: The DIAGNOSES_ICD table seems to be the most relevant one as it contains the ICD codes for diagnoses. Let's check its schema.


CREATE TABLE "DIAGNOSES_ICD" (
	row_id BIGINT, 
	subject_id BIGINT, 
	hadm_id BIGINT, 
	seq_num BIGINT, 
	icd9_code TEXT
)

/*
3 rows from DIAGNOSES_ICD table:
row_id	subject_id	hadm_id	seq_num	icd9_code
112344	10006	142345	1	99591
112345	10006	142345	2	99662
112346	10006	142345	3	5672
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT DISTINCT subject_i

{'input': 'Can you identify all subject id with sepsis',
 'output': 'The subject IDs of patients diagnosed with sepsis are: 10006, 10038, 10088, 10124, 10132, 40612, 40655, and 41976.'}

In [66]:
agent.invoke("What does diagnosis code 99591 means")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `{'table_names': 'D_ICD_DIAGNOSES'}`
responded: The table that contains the definitions for the ICD codes is D_ICD_DIAGNOSES. Let's check the schema of this table.


CREATE TABLE "D_ICD_DIAGNOSES" (
	row_id BIGINT, 
	icd9_code TEXT, 
	short_title TEXT, 
	long_title TEXT
)

/*
3 rows from D_ICD_DIAGNOSES table:
row_id	icd9_code	short_title	long_title
1	01716	Erythem nod tb-oth test	Erythema nodosum with hypersensitivity reaction in tuberculosis, tubercle bacilli not found by bacte
2	01720	TB periph lymph-unspec	Tuberculosis of periphe

{'input': 'What does diagnosis code 99591 means',
 'output': "The diagnosis code 99591 refers to 'Sepsis'."}

In [72]:
agent.invoke("identify all subject id with all types of sepsis show every icd with title containing sepsis")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


ADMISSIONS, CALLOUT, CAREGIVERS, CHARTEVENTS, CPTEVENTS, DATETIMEEVENTS, DIAGNOSES_ICD, DRGCODES, D_CPT, D_ICD_DIAGNOSES, D_ICD_PROCEDURES, D_ITEMS, D_LABITEMS, ICUSTAYS, INPUTEVENTS_CV, INPUTEVENTS_MV, LABEVENTS, MICROBIOLOGYEVENTS, NOTEEVENTS, OUTPUTEVENTS, PATIENTS, PRESCRIPTIONS, PROCEDUREEVENTS_MV, PROCEDURES_ICD, SERVICES, TRANSFERS
Invoking: `sql_db_schema` with `{'table_names': 'DIAGNOSES_ICD, D_ICD_DIAGNOSES'}`
responded: The tables DIAGNOSES_ICD and D_ICD_DIAGNOSES seem to be the most relevant for this task. DIAGNOSES_ICD contains the ICD codes for each patient and D_ICD_DIAGNOSES provides the description for each ICD code. Let's check the schema of these tables.


CREATE TABLE "DIAGNOSES_ICD" (
	row_id BIGINT, 
	subject_id BIGINT, 
	hadm_id BIGINT, 
	seq_num BIGINT, 
	icd9_code TEXT
)

/*
3 rows from DIAGNOSES_ICD table:
row_id	subject_id	hadm_id	seq_num	icd9_code
112344	10006	142345	1	9

{'input': 'identify all subject id with all types of sepsis show every icd with title containing sepsis',
 'output': "Here are some of the patients (subject_id) with ICD codes and titles containing 'sepsis':\n\n- Patient 10006 has ICD code 99591, which corresponds to 'Sepsis'.\n- Patient 10038 has ICD code 99591, which corresponds to 'Sepsis'.\n- Patient 10088 has ICD code 99591, which corresponds to 'Sepsis'.\n- Patient 10124 has ICD code 99591, which corresponds to 'Sepsis'.\n- Patient 10132 has ICD code 99591, which corresponds to 'Sepsis'.\n- Patient 40612 has ICD code 99591, which corresponds to 'Sepsis'.\n- Patient 40655 has ICD code 99591, which corresponds to 'Sepsis'.\n- Patient 41976 has ICD code 99591, which corresponds to 'Sepsis'.\n- Patient 10019 has ICD code 99592, which corresponds to 'Severe sepsis'.\n- Patient 10029 has ICD code 99592, which corresponds to 'Severe sepsis'.\n- Patient 10036 has ICD code 99592, which corresponds to 'Severe sepsis'.\n- Patient 10045 has 

### Langserve

In [73]:
!pip install -U langchain-cli

INFO: pip is looking at multiple versions of fastapi-cli to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
   --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
   ---------------------------------------- 41.5/41.5 kB 495.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/207.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/207.3 kB ? eta -:--:--
   ------- ------------------------------- 41.0/207.3 kB 487.6 kB/s eta 0:00:01
   ------- ------------------------------- 41.0/207.3 kB 487.6 kB/s eta 0:00:01
   --------------- ----------------------- 81.9/207.3 kB 456.6 kB/s eta 0:00:01
   ----------------- --------------------- 92.2/207.3 kB 403.5 kB/s eta 0:00:01
   -------------------- ----------------- 112.6/207.3 kB 435.7 kB/s eta 0:00:01
   ---------------------- --------------- 122.9/207.3 kB 423.5 kB/s eta 0:00:01
   -----

In [ ]:
from sqlalchemy import create_engine

# Create a SQLAlchemy engine to connect to the SQLite database
engine = create_engine("sqlite:///C:\\Users\\mtalh\\OneDrive\\Desktop\\ML\\Xavor\\bootcamp\\langserve_medicalmind\\mimic3.db")

In [4]:

# Connect to the engine
connection = engine.connect()

In [6]:
result = connection.execute(text("SELECT * FROM ADMISSIONS"))
result